In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Step 1: Load and Prepare Data
df = pd.read_csv('clean_close_prices_for_analysis.csv', parse_dates=['Date'])
for col in ['BND_Close', 'SPY_Close', 'TSLA_Close']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.dropna(subset=['BND_Close', 'SPY_Close', 'TSLA_Close'])
df = df.set_index('Date')

# Define backtesting period
backtest_start = '2024-08-01'
backtest_end = '2025-07-31'
df_backtest = df[backtest_start:backtest_end]

# Calculate daily returns
returns = df_backtest.pct_change().dropna()

# Step 2: Define Portfolios
# Strategy Portfolio (initial weights: 10% TSLA, 60% BND, 30% SPY)
strategy_weights = {'TSLA': 0.10, 'BND': 0.60, 'SPY': 0.30}
# Benchmark Portfolio (60% SPY, 40% BND)
benchmark_weights = {'SPY': 0.60, 'BND': 0.40}

# Step 3: Simulate Portfolio Performance
initial_capital = 10000
portfolio_value_strategy = [initial_capital]
portfolio_value_benchmark = [initial_capital]

# Daily portfolio returns
for date, daily_returns in returns.iterrows():
    # Strategy portfolio return
    strategy_return = (strategy_weights['TSLA'] * daily_returns['TSLA_Close'] +
                      strategy_weights['BND'] * daily_returns['BND_Close'] +
                      strategy_weights['SPY'] * daily_returns['SPY_Close'])
    portfolio_value_strategy.append(portfolio_value_strategy[-1] * (1 + strategy_return))

    # Benchmark portfolio return (rebalanced monthly)
    if date.day == 1:  # Simplify rebalancing to start of month
        benchmark_value = initial_capital
    benchmark_return = (benchmark_weights['SPY'] * daily_returns['SPY_Close'] +
                       benchmark_weights['BND'] * daily_returns['BND_Close'])
    portfolio_value_benchmark.append(portfolio_value_benchmark[-1] * (1 + benchmark_return))

# Convert to DataFrame for plotting
dates = df_backtest.index[1:]  # Skip the first NaN return date
portfolio_df = pd.DataFrame({
    'Strategy': portfolio_value_strategy[1:],
    'Benchmark': portfolio_value_benchmark[1:]
}, index=dates)

# Step 4: Calculate Cumulative Returns
portfolio_df['Cumulative_Return_Strategy'] = (portfolio_df['Strategy'] / initial_capital - 1) * 100
portfolio_df['Cumulative_Return_Benchmark'] = (portfolio_df['Benchmark'] / initial_capital - 1) * 100

# Step 5: Calculate Performance Metrics
# Annualized returns and volatility
daily_returns_strategy = portfolio_df['Strategy'].pct_change().dropna()
daily_returns_benchmark = portfolio_df['Benchmark'].pct_change().dropna()
annualized_return_strategy = ((portfolio_df['Strategy'][-1] / initial_capital) ** (252 / len(daily_returns_strategy))) - 1
annualized_return_benchmark = ((portfolio_df['Benchmark'][-1] / initial_capital) ** (252 / len(daily_returns_benchmark))) - 1
annualized_volatility_strategy = daily_returns_strategy.std() * np.sqrt(252)
annualized_volatility_benchmark = daily_returns_benchmark.std() * np.sqrt(252)

# Sharpe Ratio (assuming 2% risk-free rate)
sharpe_ratio_strategy = (annualized_return_strategy - 0.02) / annualized_volatility_strategy
sharpe_ratio_benchmark = (annualized_return_benchmark - 0.02) / annualized_volatility_benchmark

# Print metrics
print(f"Strategy Performance:")
print(f"Total Return: {(portfolio_df['Strategy'][-1] / initial_capital - 1) * 100:.2f}%")
print(f"Annualized Return: {annualized_return_strategy * 100:.2f}%")
print(f"Annualized Volatility: {annualized_volatility_strategy * 100:.2f}%")
print(f"Sharpe Ratio: {sharpe_ratio_strategy:.2f}")
print(f"\nBenchmark Performance:")
print(f"Total Return: {(portfolio_df['Benchmark'][-1] / initial_capital - 1) * 100:.2f}%")
print(f"Annualized Return: {annualized_return_benchmark * 100:.2f}%")
print(f"Annualized Volatility: {annualized_volatility_benchmark * 100:.2f}%")
print(f"Sharpe Ratio: {sharpe_ratio_benchmark:.2f}")

# Step 6: Plot Cumulative Returns
plt.figure(figsize=(12, 6))
plt.plot(portfolio_df.index, portfolio_df['Cumulative_Return_Strategy'], label='Strategy (10% TSLA, 60% BND, 30% SPY)')
plt.plot(portfolio_df.index, portfolio_df['Cumulative_Return_Benchmark'], label='Benchmark (60% SPY, 40% BND)')
plt.title('Cumulative Returns: Strategy vs Benchmark (Aug 2024 - Jul 2025)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return (%)')
plt.legend()
plt.grid()
plt.show()